# Estimate Options

There are several options that we can use to change or improve a given estimation.

Before we get into these, let's initialize our target IRAS 04171+2756 as a `Target`. For more information about `Target` objects, see the [Target Metadata Tutorial](./target_priors_photometry.ipynb).

In [1]:
from stelpar import Target

target = Target('IRAS 04171+2756')

## Choosing an Isochrone Model

There are three isochrone models that are built into the code, two of which are derived from the Dartmouth Stellar Evolution Program (DSEP; Chaboyer et al. 2001; DOI: [10.1086/323872](https://ui.adsabs.harvard.edu/abs/2001ApJ...562..521C/abstract); Dotter et al. 2008, DOI: [10.1086/589654](https://ui.adsabs.harvard.edu/abs/2008ApJS..178...89D/abstract)). The first is a non-magnetic DSEP model and the second is a DSEP-based model with the enhancement of magnetic field physics (Feiden & Chaboyer 2012; DOI: [10.1088/0004-637X/761/1/30](https://ui.adsabs.harvard.edu/abs/2012ApJ...761...30F/abstract)). The third model is from the PAdova and TRieste Stellar Evolution Code (PARSEC; Bressan et al. 2012, DOI: [10.1111/j.1365-2966.2012.21948.x](https://ui.adsabs.harvard.edu/abs/2012MNRAS.427..127B/abstract)).

To change the isochrone model, we pass one of `'std'` (DSEP-non-magnetic), `'mag'` (DSEP-magnetic), or `'parsec'` (PARSEC) to the `isochrone` keyword argument of an `Estimate` instance. The default is `'mag'`.

In [2]:
from stelpar import Estimate

est = Estimate(target, isochrone='std')

## Changing the Interpolation Method

There are three built-in interpolation methods available for interpolating the model grid to a given (age, mass) pair. These can be changed using the `interp_method` keyword argument in an `Estimate` instance.

- `interp_method='true'`: bilinear interpolation using `DFInterpolator` (see [docs](https://isochrones.readthedocs.io/en/latest/interpolate.html)) from the `isochrones` python package (Morton 2015; bibcode: [2015ascl.soft03010M](https://ui.adsabs.harvard.edu/abs/2015ascl.soft03010M/abstract))
- `interp_method='nearest'`: nearest-neighbor finds the closest datapoint to the given age and mass
- `interp_method='hybrid'`: finds the nearest-neighbor datapoint in age then linearly interpolates in mass

If the keywords are either `'nearest'` or `'hybrid'`, `stelpar` uses a pre-interpolated version of the isochrone model where the grid spacing is much smaller than the original model. The default is `'true'`.

In [3]:
est = Estimate(target, isochrone='std', interp_method='hybrid')


loading isochrone model grid ..


## Extinction Options

There are two optional changes we can make to the built-in extinciton calculation that is run at every iteration of the MCMC.

The first is to set $A_V=0$ and remove it entirely as a fit parameter (i.e., the calculation never happens). This is useful if we know that our target is nearby and not surrounded by gas and dust (e.g., the star is $\sim$30 pc away and does not have a protoplanetary disk nor exist in a star-forming region).

To set $A_V=0$ we pass `zero_extinction=True` to `Estimate` (the default is `False`).

In [4]:
est = Estimate(target, isochrone='std', interp_method='hybrid', zero_extinction=True)


loading isochrone model grid ..


When extinction is calculated it is done so using an identical procedure to the `synphot` python package (STScI Development Team 2018; bibcode: [2018ascl.soft11001S](https://ui.adsabs.harvard.edu/abs/2018ascl.soft11001S/abstract)), except using `numpy` arrays and `numba` instead of the built-in `synphot` object framework. This makes the calculation significantly faster. If you prefer using the pure-`synphot` method, you can pass `use_synphot=True` to `Estimate` (the default is `False`).

In [5]:
est = Estimate(target, isochrone='std', interp_method='hybrid', zero_extinction=False, use_synphot=True)


loading isochrone model grid ..


## Using Different Moves

Built into the `Target` infrastructure is the ability to use different ensemble moves. A good description of what these are and their basic usage can be found in the [`emcee` moves tutorial](https://emcee.readthedocs.io/en/stable/tutorials/moves/). All of the available moves and their references in the literature can be found in the [`emcee` moves documentation](https://emcee.readthedocs.io/en/stable/user/moves/) of the `emcee` documentation.

`stelpar` gives the ability to use four of these moves: `StretchMove`, `DEMove`, `DESnookerMove`, and `KDEMove`, all from `emcee`. They can be accessed via the keywords `"stretch"`, `"DE"`, `"snooker"`, and `"KDE"`, respectively. Along with a keyword, a weight must be applied to each move. Any arguments can also be given to each move function as you would with `emcee`. 

`StretchMove` with a weight of 1.0 is the default move.

For example, suppose we want to use 20% `DEMove` and 80% `DESnookerMove`:

In [6]:
target.moves = [
    ('snooker', 0.8), 
    ('DE', 0.2),
]

target.moves

[(<emcee.moves.de_snooker.DESnookerMove at 0x1ed2dba7260>, 0.8),
 (<emcee.moves.de.DEMove at 0x1ed2da32930>, 0.2)]